In [19]:
import copy as copy
import numpy as np
import random

def printState(state):
    stateCopy = copy.deepcopy(state)
    maxLength = findLongest(stateCopy)
    for item in stateCopy:
        for x in range (len(item) , maxLength):
            item.insert(0, ' ')

    for i in range(maxLength):
        for j in range(len(stateCopy)):
            print("  {}".format(stateCopy[j][i]), end='')
        print()
    print('--|--|--|--\n\n')


def findLongest(state):
    length = 0
    for item in state:
        length = max(length, len(item))

    return length


def validMoves(state):
    validStates = []

    for itemToMove in state:
        # Check if the list is empty
        if itemToMove:
            # Iterate through the rest of the states
            for placeToMove in range(len(state)):
                # don't count as valid move if it's back to the same location
                if state.index(itemToMove) != placeToMove:
                    # valid move if the place to move is empty or the item to move is smaller
                    if len(state[placeToMove]) == 0 or itemToMove[0] < state[placeToMove][0]:
                        # append the tuple (move from, move to) to the valid states
                        validStates.append([state.index(itemToMove)+1, placeToMove+1])
    return validStates


def makeMove(state, move):
    item = state[move[0]-1].pop(0)
    state[move[1]-1].insert(0, item)
    return state


def unMakeMove(state, move):
    item = state[move[1]-1].pop(0)
    state[move[0]-1].insert(0, item)
    return state


def winner(state):
    board = [[], [], [1,2,3]]
    return state == board

def myTupler(state):
    '''Need immutable type for key to dictionary '''
    superTuple = tuple(tuple(s) for s in state)
    return superTuple


def epsilonGreedy(epsilon, Q, state, validMovesF):
    goodMoves = validMovesF(state)
    if np.random.uniform() < epsilon:
        # Random Move
        return tuple(random.choice(goodMoves))
    else:
        # Greedy Move
        Qs = np.array([Q.get((myTupler(state),tuple(m)), 0) for m in goodMoves])
        return tuple(goodMoves[np.argmax(Qs)])


def trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF):
    maxGames = nRepetitions
    rho = learningRate
    epsilonDecayRate = epsilonDecayFactor
    epsilon = 1.0
    Q = {}
    stepList = []
    showMoves = False

    for nGames in range(maxGames):
        epsilon *= epsilonDecayRate
        step = 0
        state = [[1, 2, 3], [], []]
        done = False

        while not done:
            step += 1
            move = epsilonGreedy(epsilon, Q, state, validMovesF)
            stateNew = copy.deepcopy(state)
            makeMoveF(stateNew, move)
            if (myTupler(state), move) not in Q:
                Q[(myTupler(state), move)] = 0.0  # Initial Q value for new state, move
            if showMoves:
                printState(stateNew)
            if winner(stateNew):
                # We won!  backfill Q
                if showMoves:
                    print('End State, we won!')
                Q[(myTupler(state), move)] = 1.0
                done = True
                stepList.append(step)

            if step > 1:
                #Q[(myTupler(stateOld), moveOld)] += rho * (Q[(myTupler(state), move)] - Q[(myTupler(stateOld), moveOld)])
                Q[(myTupler(stateOld), moveOld)] += rho * (Q[(myTupler(state), move)] - Q[(myTupler(stateOld), moveOld)])

            stateOld, moveOld = state, move
            state = stateNew

    print(epsilon)
    return Q,stepList

In [33]:
initialState = [[1,2,3],[],[]]
initialState = [[1],[2],[3]]
print(validMoves(initialState))
myState = [[1 ,2] ,[3] ,[]]
printState(myState)

[(0, 1), (0, 2), (1, 2)]
  1      
  2  3    
--|--|--|--




In [18]:
for move in validMoves(myState):
    makeMove(myState, move)
    printState(myState)
    unMakeMove(myState, move)
    
print(len(validMoves(myState)))

NameError: name 'myState' is not defined

In [50]:
Q = {}
def myTupler(state):
    '''Need immutable type for key to dictionary '''
    superTuple = tuple(tuple(l) for l in state)
    return superTuple
    
Q[myTupler(initialState),(0,1)] = 0
Q

{(((1,), (2,), (3,)), (0, 1)): 0}

In [53]:
rho = 0.1
import random
import numpy as np
# move = random.choice(validMoves(myState))
move = (0,2)
print(validMoves(initialState))
print(Q.get((myTupler(initialState),move),0))

Q[(myTupler(initialState),move)] = Q.get((myTupler(initialState),move),0) + rho * (-1 - Q.get((myTupler(initialState),move),0))
Q

[[1, 2], [1, 3], [2, 3]]
-0.19


{(((1,), (2,), (3,)), (0, 1)): 0, (((1,), (2,), (3,)), (0, 2)): -0.271}

In [58]:
Qs = np.empty(len(validMoves(initialState)))
print(initialState)
Qs = np.array((myTupler(initialState),m),0) for m in validMoves(initialState))
print('Q values for validMoves are',Qs)
print(Qs)

SyntaxError: invalid syntax (<ipython-input-58-2f92837378e5>, line 3)

In [24]:
%run -i A5grader.py


Testing validMoves([[1], [2], [3]])

--- 10/10 points. Correctly returned [[1, 2], [1, 3], [2, 3]]

Testing validMoves([[], [], [1, 2, 3]])

--- 10/10 points. Correctly returned [[3, 1], [3, 2]]

Testing makeMove([[], [], [1, 2, 3]], [3, 2])

--- 10/10 points. Correctly returned [[], [1], [2, 3]]

Testing makeMove([[2], [3], [1]], [1, 2])

--- 10/10 points. Correctly returned [[], [2, 3], [1]]

Testing   Q, steps = trainQ(1000, 0.5, 0.7, validMoves, makeMove).
1.253256639965641e-155

---  0/10 points. Q dictionary should have close to 76 entries. Yours has 65

---  0/10 points. The mean of the number of steps is incorrect.  Yours is 10.451.  It should be less than 10.

Testing   path = testQ(Q, 20, validMoves, makeMove).

--- testQ raised the exception
 name 'testQ' is not defined

A5 Execution Grade is 40/80

 Remaining 20 points will be based on your text describing the trainQ and test! functions.

A5 FINAL GRADE is __/100
